<a href="https://colab.research.google.com/github/C-Little26/ECGR4105/blob/main/Hw5/ECGR4105_Hw5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#necessary libraries
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
import torch
import tensorflow
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filepath = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(filepath))
housing.isnull().sum()*100/housing.shape[0] #use to check for null values
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [5]:
#define the function to remap the variables
def binary_map(x):
    return x.map({"yes": 1, "no": 0})
map_vars = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea']
housing[map_vars] = housing[map_vars].apply(binary_map)

In [8]:
#establish matrices
Y = housing.pop('price')
X = housing.drop('furnishingstatus',axis=1)
#train/test split
x_train, x_val, y_train, y_val = train_test_split(X,Y, train_size = 0.8, test_size = 0.2, random_state=0)

In [9]:
#standardize values
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_val = sc_x.fit_transform(x_val)

In [10]:
#stack arrays before conversion
x_t_arr = np.c_[np.ones((len(y_train), 1)), x_train]
x_v_arr = np.c_[np.ones((len(y_val),1)), x_val]

#convert to tensors
x_t_ten = torch.tensor(x_t_arr)
x_v_ten = torch.tensor(x_v_arr)
y_t_ten = torch.tensor(y_train.values)
y_v_ten = torch.tensor(y_val.values)
#have to call .values for it to be treated like an array

In [11]:
#define the model
def model(t_u, w12, w11, w10, w9, w8, w7, w6, w5, w4, w3, w2, w1, b):
         return w12*t_u**12 + w11*t_u**11 + w10*t_u**10 + w9*t_u**9 + w8*t_u**8 + w7*t_u**7 + w6*t_u**6 + w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2*t_u**2 + w1*t_u + b

#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [12]:
#define the training loop
def loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params

In [17]:
#establish variables for model iterations
learning_rate = [1e-15, 1e-16, 1e-17, 1e-18]
sgd_params = torch.zeros(len(learning_rate), 13)
sgd_loss = torch.zeros(len(learning_rate), 1)
#run training for each learning rate using SGD
for i in learning_rate:
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
    optimizer = optim.SGD([params], lr=i)
    print(f'\nTraining with learning rate: {i}')
    sgd_params[learning_rate.index(i)] = loop(
        n_epochs = 5000, optimizer = optimizer,
        params = params,
        t_u_train = x_t_ten, t_u_val = x_v_ten,
        t_c_train = y_t_ten, t_c_val = y_v_ten, )


Training with learning rate: 1e-15
Epoch 1, Training loss: 319201294472925.1250, Validation loss: 56796102370111.3281

Epoch 2, Training loss: 129047067691029.5469, Validation loss: 35605577863566.2109

Epoch 3, Training loss: 62344543726830.4062, Validation loss: 28409800615645.2344

Epoch 500, Training loss: 26304279573533.5156, Validation loss: 25052951615431.7539

Epoch 1000, Training loss: 26304232491507.4023, Validation loss: 25052769869402.7969

Epoch 1500, Training loss: 26304185546289.6836, Validation loss: 25052588288663.5938

Epoch 2000, Training loss: 26304138699587.4023, Validation loss: 25052406763032.0312

Epoch 2500, Training loss: 26304092117382.7227, Validation loss: 25052226245144.7031

Epoch 3000, Training loss: 26304045796900.7070, Validation loss: 25052046252713.5898

Epoch 3500, Training loss: 26303999627682.0547, Validation loss: 25051866482595.4805

Epoch 4000, Training loss: 26303953737923.4297, Validation loss: 25051687709147.1562

Epoch 4500, Training loss:

In [19]:
#establish variables for model iterations
learning_rate = [0.1,0.01,0.001,0.0001]
adam_params = torch.zeros(len(learning_rate), 13)
adam_loss = torch.zeros(len(learning_rate), 1)
#run training for each learning rate using Adam
for i in learning_rate:
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
    optimizer = optim.Adam([params], lr=i)
    print(f'\nTraining with learning rate: {i}')
    adam_params[learning_rate.index(i)] = loop(
        n_epochs = 5000, optimizer = optimizer,
        params = params,
        t_u_train = x_t_ten, t_u_val = x_v_ten,
        t_c_train = y_t_ten, t_c_val = y_v_ten, )


Training with learning rate: 0.1
Epoch 1, Training loss: 319201294472925.1250, Validation loss: 56796102370111.3281

Epoch 2, Training loss: 262373642457297.2188, Validation loss: 50382984220773.6953

Epoch 3, Training loss: 211871892236607.7812, Validation loss: 44715298728724.4922

Epoch 500, Training loss: 26317684835564.0820, Validation loss: 25097434006374.5000

Epoch 1000, Training loss: 26308862034727.0508, Validation loss: 25079114085203.6523

Epoch 1500, Training loss: 26297729271239.8398, Validation loss: 25054917570665.9258

Epoch 2000, Training loss: 26285233097353.8789, Validation loss: 25026414487972.8984

Epoch 2500, Training loss: 26272250174742.3359, Validation loss: 24995294436062.8828

Epoch 3000, Training loss: 26259476642742.0977, Validation loss: 24963410763241.5430

Epoch 3500, Training loss: 26247223031546.5078, Validation loss: 24932561730383.4609

Epoch 4000, Training loss: 26235272508448.2148, Validation loss: 24904137369902.3945

Epoch 4500, Training loss: 